In [3]:
#this defaults plots to matplotlib 1.0 formatting
import matplotlib.style
import matplotlib as mpl
#mpl.style.use('classic')
%matplotlib inline
mpl.rcParams['figure.dpi']= 150

# importing packages needed to run this script 
from subprocess import call
import pickle as pickle
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.cluster.hierarchy as sch

import os
import time
import matplotlib.pyplot as plt
import numpy as np
import copy
import colorsys
import requests
import matplotlib.image as mpimg
from IPython.display import display
from IPython.display import Image
import scipy.cluster.hierarchy as sch
from scipy.stats import scoreatpercentile 
# import mpld3
import pickle as pickle
from optparse import OptionParser
import sys
sys.path.append('../../utilities')
sys.path.append('..')
from pysca import scaTools as sca
from graph_helpers import (
   gen_amino_acid_position_in_ref_sequence_graph,
   gen_clustered_similarity_matrix, 
   gen_unclustered_similarity_matrix,
   gen_pairwise_sequence_identities
)
from helpers import (
    show_processmsa_results,
    seq_count_check
)
from valine_finder import (
    get_valine_header, #given header string, pulls out name of species
    get_family_string, #given header string, pulls out lineage
    get_valine_positions, #given sequence and positions, Identifies character at each position
    valine_finder #given file and position returns dict of species name:{lineage,positions}
)

%matplotlib inline

In [33]:
#open db file
dbname = "HMGCR_cho_alignment.db"
pickle_path = './HMGCR_cho_alignment.db'
db = pickle.load(open(pickle_path,'rb'))
Dseq = db['sequence']
Dsca = db['sca']
Dsect = db['sector'] if db['sector'] else None

In [34]:
ats = Dseq['ats']
alg = Dseq['alg']
hd = Dseq['hd']
positions = [230,234] #--252,256-- 231,235 shifted for indexing

In [35]:
rows = []
for i, h in enumerate(Dseq['hd']):
    rows.append({get_valine_header(h):{"lineage":get_family_string(h),"notable_positions":get_valine_positions(alg[i],positions)}})
print(rows)

[{'Cricetulus griseus (Chinese hamster)': {'lineage': 'Eukaryota, Metazoa, Chordata, Craniata, Vertebrata, Euteleostomi, Mammalia, Eutheria, Euarchontoglires, Glires, Rodentia, Myomorpha, Muroidea, Cricetidae, Cricetinae, Cricetulus', 'notable_positions': ['V', 'V']}}, {'Amazona collaria (yellow-billed parrot)': {'lineage': 'Eukaryota, Metazoa, Chordata, Craniata, Vertebrata, Euteleostomi, Archelosauria, Archosauria, Dinosauria, Saurischia, Theropoda, Coelurosauria, Aves, Neognathae, Psittaciformes, Psittacidae, Amazona', 'notable_positions': ['V', 'V']}}, {'Accipiter nisus (Eurasian sparrowhawk)': {'lineage': 'Eukaryota, Metazoa, Chordata, Craniata, Vertebrata, Euteleostomi, Archelosauria, Archosauria, Dinosauria, Saurischia, Theropoda, Coelurosauria, Aves, Neognathae, Accipitriformes, Accipitridae, Accipitrinae, Accipiter', 'notable_positions': ['V', 'V']}}, {'Accipiter nisus (Eurasian sparrowhawk)': {'lineage': 'Eukaryota, Metazoa, Chordata, Craniata, Vertebrata, Euteleostomi, Arche

In [38]:
#write that to a csv
output_file = open("./HMGCR_cho-notable-valines.csv", "w")
output_file.write("Common Name, Lineage")
for position in positions:
    output_file.write(f",Notable Valine at Position {position+1}")
output_file.write('\n')
for row in rows:
        keys = list(row.keys())
        common_name = keys[0]
        nested_obj = row.get(common_name)
        lineage = nested_obj.get('lineage').replace(",", "->").strip("\n")
        output_file.write(f"{common_name}, {lineage},{','.join(nested_obj.get('notable_positions'))}")
        output_file.write('\n')
output_file.close()